**INFER/RECOMMEND SIZE FOR A CUSTOMER**

For this analysiss we have only considered one of the two of the dataset

This part of task was worked out together by all 3 of us equally. 

The preprocessing part was taken up from the already existing preprocessing file 'Fit-Prediction-Preprocessing-And-Merging-Dataset.ipynb'

In [0]:
from google.colab import drive
drive.mount('/content/drive')

https://seaborn.pydata.org/index.html

https://matplotlib.org/

In [0]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
# Train and test split
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

In [0]:
data = pd.read_json('/content/drive/My Drive/Colab Notebooks/256-project/renttherunway_final_data.json', lines=True)

In [4]:
data.head()

,fit,user_id,bust size,item_id,weight,rating,rented for,review_text,body type,review_summary,category,height,size,age,review_date
0,fit,420272,34d,2260466,137lbs,10.0,vacation,An adorable romper! Belt and zipper were a lit...,hourglass,So many compliments!,romper,"5' 8""",14,28.0,"April 20, 2016"
1,fit,273551,34b,153475,132lbs,10.0,other,I rented this dress for a photo shoot. The the...,straight & narrow,I felt so glamourous!!!,gown,"5' 6""",12,36.0,"June 18, 2013"
2,fit,360448,NaN,1063761,NaN,10.0,party,This hugged in all the right places! It was a ...,NaN,It was a great time to celebrate the (almost) ...,sheath,"5' 4""",4,116.0,"December 14, 2015"
3,fit,909926,34c,126335,135lbs,8.0,formal affair,I rented this for my company's black tie award...,pear,Dress arrived on time and in perfect condition.,dress,"5' 5""",8,34.0,"February 12, 2014"
4,fit,151944,34b,616682,145lbs,10.0,wedding,I have always been petite in my upper body and...,athletic,Was in love with this dress !!!,gown,"5' 9""",12,27.0,"September 26, 2016"


In [5]:
data.corr()

,user_id,item_id,rating,size,age
user_id,1.000000,-0.002721,-0.006973,0.003442,0.007424
item_id,-0.002721,1.000000,-0.001163,-0.044311,0.026516
rating,-0.006973,-0.001163,1.000000,-0.036731,-0.035280
size,0.003442,-0.044311,-0.036731,1.000000,0.155687
age,0.007424,0.026516,-0.035280,0.155687,1.000000


Selecting only the features that might be relevent for the Size inference

In [0]:
dfRent = data[['weight','height','body type','size','fit','age','category','bust size']]

In [7]:
dfRent.head(3)

,weight,height,body type,size,fit,age,category,bust size
0,137lbs,"5' 8""",hourglass,14,fit,28.0,romper,34d
1,132lbs,"5' 6""",straight & narrow,12,fit,36.0,gown,34b
2,NaN,"5' 4""",NaN,4,fit,116.0,sheath,NaN


In [8]:
dfRent.isnull().mean().round(4) * 100

weight       15.57
height        0.35
body type     7.60
size          0.00
fit           0.00
age           0.50
category      0.00
bust size     9.56
dtype: float64

In [9]:
dfRent.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 192544 entries, 0 to 192543
Data columns (total 8 columns):
weight       162562 non-null object
height       191867 non-null object
body type    177907 non-null object
size         192544 non-null int64
fit          192544 non-null object
age          191584 non-null float64
category     192544 non-null object
bust size    174133 non-null object
dtypes: float64(1), int64(1), object(6)
memory usage: 11.8+ MB


**BELOW PREPROCESSING FOLLOWED SIMILAR TO THE PREPROCESSING IN FILE 'Fit-Prediction-Preprocessing-And-Merging-Dataset.ipynb'**

**values in 'height' column can be converted to cms**
****

In [0]:
def get_height_cms(height):
    if type(height) == type(1.0):
        return
    try: 
        h_inch = int(height[3:-1])
        h_inch += int(height[0]) * 12
        h_cm = round(h_inch * 2.54, 1)        
        return h_cm
    except:
        return (int(height[0]) * 12 * 2.54)

In [0]:
dfRent['height'] = dfRent['height'].apply(get_height_cms)

In [0]:
def get_weight(weight):
    if type(weight) == type(1.0):
        return
    weightN = int(weight[0:-3])        
    return weightN

dfRent['weight'] = dfRent['weight'].apply(get_weight)

In [0]:
dfRent['weight'] = dfRent['weight'].astype('float')

**Impute the missing values for 'weight' column with mean**

In [0]:
from sklearn.impute import SimpleImputer
weightImp = np.array(dfRent['weight']).reshape(-1, 1)
imp = SimpleImputer(strategy='mean')
dfRent['weight'] = imp.fit_transform(weightImp)

**The 'height' column has 0.65% of missing values we can impute the missing values by considering the mean factor**

In [0]:
# from sklearn.impute import SimpleImputer
heightImp = np.array(dfRent['height']).reshape(-1, 1)
imp = SimpleImputer(strategy='median')
dfRent['height'] = imp.fit_transform(heightImp)

In [0]:
dfRent['body type'].fillna('Unspecified', inplace=True)
dfRent['body type'] = dfRent['body type'].astype('category')

In [0]:
from sklearn import preprocessing
labelEnc = preprocessing.LabelEncoder()
dfRent['body type'] = labelEnc.fit_transform(dfRent['body type'])

In [0]:
dfRent['category'] = dfRent['category'].astype('category')
dfRent['category'] = labelEnc.fit_transform(dfRent['category'])

In [0]:
dfRent['bust size'].fillna('Unspecified', inplace=True)
dfRent['bust size'] = dfRent['bust size'].astype('category')
dfRent['bust size'] = labelEnc.fit_transform(dfRent['bust size'])

In [0]:
dfRent['fit'] = dfRent['fit'].astype('category')

In [0]:
def fit_to_numeric(fit):
    if fit=='small':
        return 1
    if fit=='fit':
        return 2
    if fit=='large':
        return 3

dfRent['fit_to_numeric'] = dfRent['fit'].apply(fit_to_numeric)

In [23]:
dfRent.head()

,weight,height,body type,size,fit,age,category,bust size,fit_to_numeric
0,137.000000,172.7,4,14,fit,28.0,44,39,2
1,132.000000,167.6,7,12,fit,36.0,20,37,2
2,137.391709,162.6,0,4,fit,116.0,45,106,2
3,135.000000,165.1,5,8,fit,34.0,16,38,2
4,145.000000,175.3,2,12,fit,27.0,20,37,2


**TO PROCEED WITH THIS IMPLEMENTATION WE ARE CONSIDERING ONLY THE "FIT" DATA OF THE CUSTOMER AS PART OF TRAINING THE MODEL AND LEARN THE FIT FOR EVERY INSTANCE/CUSTOMER BASED ON THE ATTRIBUTES SPECIFIED BY THE USER TO PREDICT THE SIZE**

In [0]:
dfRent = dfRent[dfRent['fit_to_numeric'] == 2]

In [0]:
dfRent = dfRent.drop(columns={'fit_to_numeric','fit'},axis=1)

In [26]:
dfRent.shape

(142058, 7)

**'age' feature impute the missing values with mean**

In [0]:
ageImp = np.array(dfRent['age']).reshape(-1, 1)
imp = SimpleImputer(strategy='mean')
dfRent['age'] = imp.fit_transform(ageImp)

In [0]:
dfRent['age'] = dfRent['age'].astype('int')

In [29]:
dfRent.describe()

,weight,height,body type,size,age,category,bust size
count,142058.000000,142058.000000,142058.000000,142058.000000,142058.000000,142058.000000,142058.000000
mean,137.301627,165.911150,3.663954,12.097249,33.886842,23.924355,45.665531
std,20.073273,6.753198,1.897160,8.400862,8.083542,12.678789,22.516600
min,50.000000,137.200000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,125.000000,160.000000,2.000000,8.000000,29.000000,16.000000,35.000000
50%,137.391709,165.100000,4.000000,12.000000,32.000000,20.000000,38.000000
75%,145.000000,170.200000,5.000000,16.000000,37.000000,26.000000,51.000000
max,300.000000,198.100000,7.000000,58.000000,117.000000,67.000000,106.000000


In [0]:
df = dfRent

In [31]:
df.head()

,weight,height,body type,size,age,category,bust size
0,137.000000,172.7,4,14,28,44,39
1,132.000000,167.6,7,12,36,20,37
2,137.391709,162.6,0,4,116,45,106
3,135.000000,165.1,5,8,34,16,38
4,145.000000,175.3,2,12,27,20,37


In [32]:
df.isnull().mean().round(4) * 100

weight       0.0
height       0.0
body type    0.0
size         0.0
age          0.0
category     0.0
bust size    0.0
dtype: float64

In [33]:
dfRent.head()

,weight,height,body type,size,age,category,bust size
0,137.000000,172.7,4,14,28,44,39
1,132.000000,167.6,7,12,36,20,37
2,137.391709,162.6,0,4,116,45,106
3,135.000000,165.1,5,8,34,16,38
4,145.000000,175.3,2,12,27,20,37


In [34]:
dfRent.corr()

,weight,height,body type,size,age,category,bust size
weight,1.000000,0.354683,-0.199245,0.740676,0.062525,-0.016388,0.304074
height,0.354683,1.000000,-0.118369,0.235753,-0.007602,-0.005913,0.085782
body type,-0.199245,-0.118369,1.000000,-0.185946,-0.012738,0.000768,-0.236510
size,0.740676,0.235753,-0.185946,1.000000,0.157633,-0.010183,0.371962
age,0.062525,-0.007602,-0.012738,0.157633,1.000000,0.022846,0.073451
category,-0.016388,-0.005913,0.000768,-0.010183,0.022846,1.000000,-0.011591
bust size,0.304074,0.085782,-0.236510,0.371962,0.073451,-0.011591,1.000000


**BUILDING THE TRAIN TEST SET**

In [0]:
dfTrain = df

In [36]:
dfTrain.shape

(142058, 7)

In [37]:
dfTrain.head()

,weight,height,body type,size,age,category,bust size
0,137.000000,172.7,4,14,28,44,39
1,132.000000,167.6,7,12,36,20,37
2,137.391709,162.6,0,4,116,45,106
3,135.000000,165.1,5,8,34,16,38
4,145.000000,175.3,2,12,27,20,37


In [0]:
train  = dfTrain.drop(columns={'size'})

In [39]:
train.shape

(142058, 6)

In [0]:
target = dfTrain[['size']]

In [41]:
target.shape

(142058, 1)

In [0]:
X_train, X_test, y_train, y_test = train_test_split(train, target, test_size=0.2, random_state=1)

In [45]:
X_train.shape

(113646, 6)

In [46]:
X_test.shape

(28412, 6)

In [47]:
y_train.shape

(113646, 1)

In [48]:
y_test.shape

(28412, 1)

Apply KNN to train the model using gridsearch to get the optimal parameter

In [0]:
from sklearn.model_selection import GridSearchCV
param_grid = { 'n_neighbors':[10,20,30,50],
               'algorithm': ['brute','auto'],
               'weights': ['uniform','distance']}

In [51]:
gs = GridSearchCV(KNeighborsClassifier(), param_grid, cv=3)
gs.fit(X_train,y_train)
gs.best_params_

{'algorithm': 'auto', 'n_neighbors': 50, 'weights': 'distance'}

In [52]:
modelKNN = KNeighborsClassifier(n_neighbors=50,weights='distance')
modelKNN.fit(X_train,y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=50, p=2,
                     weights='distance')

In [0]:
inferSize = modelKNN.predict(X_test)

In [54]:
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
print("Accuracy: ",round(((modelKNN.score(X_test,y_test))*100),2),"%")

Accuracy:  38.84 %


**Though the accuracy is only 38%, its is actually not relevant in determining the performance of the problem definition because we just want to infer the probable size for the customer...**

**Create new dataframe to store the result**

In [0]:
printData = X_test
printData['size'] = np.array(y_test)
printData['inferred_size'] = np.array(inferSize)

In [59]:
printData.head()

,weight,height,body type,age,category,bust size,size,inferred_size
11186,110.0,157.5,0,22,20,106,1,8
133060,130.0,165.1,2,45,20,24,8,8
58545,145.0,165.1,3,30,16,41,16,16
67461,130.0,157.5,2,31,16,106,8,8
182173,114.0,154.9,4,34,45,39,8,4


In [61]:
data = printData.groupby(['category','body type','size']).first()
data = pd.DataFrame(data)
data.head(30)

weight  height  age  bust size  inferred_size
category body type size                                                   
0        3         42    137.391709   170.2   23         76             24
         4         20    149.000000   162.6   49         38             14
1        0         1     137.391709   157.5   28        106              8
                   4     137.391709   160.0   31        106              8
                   8     137.391709   175.3   31        106              8
                   12    126.000000   167.6   21         37              8
                   16    137.391709   162.6   36         52             14
         1         12    135.000000   170.2   32         37              8
                   20    137.391709   175.3   40         64             20
         2         1     120.000000   167.6   26         25              4
                   4     136.000000   170.2   35         25              8
                   8     115.000000   162.6   32         37              4
                   12    135.000000   172.7   28         26              8
                   14    125.000000   165.1   33         48              8
                   16    125.000000   167.6   51         35              8
                   20    161.000000   172.7   32        106             20
         3         4     115.000000   162.6   49         38              4
                   8     137.391709   160.0   30        106              8
                   12    127.000000   162.6   30         52              8
                   16    150.000000   160.0   29         41             14
                   20    180.000000   172.7   38         52             24
                   24    165.000000   167.6   34        106             20
         4         1     117.000000   162.6   35         37              4
                   4     130.000000   170.2   31         38              8
                   8     150.000000   177.8   38         38             14
                   12    140.000000   160.0   43         39              8
                   14    140.000000   170.2   31         26              8
                   16    155.000000   180.3   32         50             14
                   20    175.000000   170.2   35         52             20
         5         1     118.000000   167.6   27         26              4

**Model is not very accurate as there is variation for many instances between the size purchased and inferred size**